[View in Colaboratory](https://colab.research.google.com/github/farahbakhsh3/X1/blob/master/New.ipynb)

In [0]:
!apt install unrar
!wget https://www.dropbox.com/s/rl6nx5e5h0ff4ub/lesson9.rar
!unrar x -r lesson9.rar

In [0]:
import random
 
import numpy as np
import tensorflow.keras as keras
 
 
# load data (raw text file)
################################################################################
################################################################################
 
path = './lesson9/data/ferdosi.txt'
with open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))
 
# create two dictionaries: (index to id) & (id to index)
################################################################################
################################################################################
 
chars = sorted(list(set(text)))
len_of_char = len(chars)
print('total chars:', len(chars))
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))
 
# convert each character to an unique id
################################################################################
################################################################################
 
max_length = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
print('nb sequences:', len(sentences))
 
# create proper vector for training model
################################################################################
################################################################################
print('Vectorization...')
x = np.zeros((len(sentences), max_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char2idx[char]] = 1
    y[i] = char2idx[next_chars[i]]
 
 
# two helper functions to print generated text at the end of each epoch
################################################################################
################################################################################
def on_epoch_end(epoch, _):
    if epoch % 10 != 0:
        return
    
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
 
    start_index = random.randint(0, len(text) - max_length - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)
 
        generated = ''
        sentence = text[start_index: start_index + max_length]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
 
        for i in range(400):
            x_pred = np.zeros((1, max_length, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char2idx[char]] = 1.
 
            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_char = idx2char[pred_id]
 
            generated += next_char
            sentence = sentence[1:] + next_char
 
        print(generated)
 
 
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
 
 
# Define simple GRU model for generating text
# In case of using CPU replace the CuDNNGRU with GRU
################################################################################
################################################################################
 
model = keras.Sequential()
model.add(keras.layers.CuDNNGRU(256, input_shape=(max_length, len_of_char)))
# model.add(keras.layers.GRU(256, input_shape=(max_length, len_of_char)))
model.add(keras.layers.Dense(len_of_char, activation='softmax'))
 
# Create graph and train the model for 60 epoch
################################################################################
################################################################################
 
model.compile(optimizer=keras.optimizers.RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
print(model.summary())
print_callback = keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback])

corpus length: 2553468
total chars: 47
nb sequences: 851143
Vectorization...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnngru_1 (CuDNNGRU)       (None, 256)               234240    
_________________________________________________________________
dense_1 (Dense)              (None, 47)                12079     
Total params: 246,319
Trainable params: 246,319
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/60
850688/851143 [============================>.] - ETA: 0s - loss: 1.8484 - acc: 0.4630
----- Generating text after Epoch: 0
----- diversity: 1.0
----- Generating with seed: "یزدان نیکی دهش کرد یاد
بفرمود کان را که "
یزدان نیکی دهش کرد یاد
بفرمود کان را که دارگفت
چو دادن و چندان گسلاستست
بدو پیز شد بر نیازور دشمن کاست شاد
که چون روی نده انکمانان چنین
پسیرام شاهگرد زندی به یزدانشوی
ز کاران بگفت او فریبرز گون
بدینی گرین آودی پرندی
به مغز ب